<a href="https://colab.research.google.com/github/supunabeywickrama/my-colab-work/blob/main/Sahi_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U ultralytics sahi opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88


In [2]:
from google.colab import files

uploaded = files.upload()


Saving best.pt to best.pt
Saving crowdv2.mp4 to crowdv2.mp4


In [3]:
!ls /content


best.pt  crowdv2.mp4  sample_data


In [4]:
from sahi.models.ultralytics import UltralyticsDetectionModel

detection_model = UltralyticsDetectionModel(
    model_path="/content/best.pt",
    confidence_threshold=0.25,
    device="cuda"
)

print("Model loaded")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Model loaded


In [5]:
import cv2
import numpy as np
from sahi.predict import get_sliced_prediction

video_path = "/content/crowdv2.mp4"
out_path = "/content/sahi_output.mp4"

cap = cv2.VideoCapture(video_path)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 25

out = cv2.VideoWriter(
    out_path,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (w, h)
)

frame_id = 0
written = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    if frame_id % 2 != 0:   # 🔽 process every 2nd frame
        continue

    result = get_sliced_prediction(
        image=frame,
        detection_model=detection_model,
        slice_height=512,
        slice_width=512,
        overlap_height_ratio=0.3,
        overlap_width_ratio=0.3
    )

    for obj in result.object_prediction_list:
        x1, y1, x2, y2 = map(int, obj.bbox.to_xyxy())
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

    out.write(np.ascontiguousarray(frame))
    written += 1

cap.release()
out.release()

print("Frames written:", written)
print("Saved:", out_path)


Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 slices.
Performing prediction on 66 

In [6]:
!ffmpeg -i /content/sahi_output.mp4 -vcodec libx264 -crf 28 /content/sahi_output_small.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [7]:
!ls -lh /content/sahi_output*


-rw-r--r-- 1 root root 773M Dec 21 12:03 /content/sahi_output.mp4
-rw-r--r-- 1 root root 104M Dec 21 12:10 /content/sahi_output_small.mp4


In [8]:
files.download("/content/sahi_output_small.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
files.download("/content/sahi_output.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>